In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from collections import Counter

from encodeNorm1 import encodeNorm
from preproc1 import na, preprocessing
from alg1 import model

In [2]:
DATA='ugrin2020-vehiculo-usado-multiclase/'
TRAIN=DATA+'train.csv'
TEST=DATA+'test.csv'

RESULTS='results/'

In [3]:
train = pd.read_csv(TRAIN) # Cargo datos de entrenamiento
test = pd.read_csv(TEST) # Cargo datos de test

# Eliminamos el campo id ya que no se debe usar para predecir
test_ids = test['id']
del test['id']
del train['id']

# Cambiamos el nombre a la columna Año para poder manejarla correctamente
train.rename(columns = {'Año':'Anio'}, inplace = True)
test.rename(columns = {'Año':'Anio'}, inplace = True)

In [4]:
Counter(train.Precio_cat) # Clases desbalanceadas

Counter({1: 269, 2: 602, 3: 2211, 4: 978, 5: 759})

In [5]:
train

,Nombre,Ciudad,Anio,Kilometros,Combustible,Tipo_marchas,Mano,Consumo,Motor_CC,Potencia,Asientos,Descuento,Precio_cat
0,Maruti Swift Dzire ZDI,G,2012.0,83000.0,Diesel,Manual,First,23.4 kmpl,1248 CC,74 bhp,5.0,NaN,3
1,Maruti Wagon R LXI Optional,I,2016.0,4800.0,Petrol,Manual,First,20.51 kmpl,998 CC,67.04 bhp,5.0,NaN,3
2,Mahindra KUV 100 mFALCON D75 K8,F,2016.0,26000.0,Diesel,Manual,First,25.32 kmpl,1198 CC,77 bhp,6.0,NaN,3
3,Hyundai i20 1.2 Magna,E,2013.0,56127.0,Petrol,Manual,First,18.5 kmpl,1197 CC,80 bhp,5.0,NaN,3
4,Honda Jazz 1.2 SV i VTEC,H,2017.0,41981.0,Petrol,Manual,First,18.7 kmpl,1199 CC,88.7 bhp,5.0,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4814,Maruti Swift VDI,E,2014.0,83274.0,Diesel,Manual,First,22.9 kmpl,1248 CC,74 bhp,5.0,NaN,3
4815,Porsche Panamera Diesel 250hp,H,2014.0,60033.0,Diesel,Automatic,First,18.18 kmpl,2967 CC,250 bhp,4.0,NaN,5
4816,Hyundai Creta 1.6 CRDi SX Option,C,2017.0,40158.0,Diesel,Manual,First,19.67 kmpl,NaN,126.2 bhp,5.0,NaN,4
4817,Hyundai Xcent 1.2 VTVT S,E,2015.0,65743.0,Petrol,Manual,First,20.14 kmpl,1197 CC,81.86 bhp,5.0,7.85,3


In [6]:
print(train.isna().sum())
print()
print(test.isna().sum())

Nombre            72
Ciudad            72
Anio              72
Kilometros        72
Combustible       72
Tipo_marchas      72
Mano              72
Consumo           73
Motor_CC         101
Potencia         175
Asientos         106
Descuento       4160
Precio_cat         0
dtype: int64

Nombre             0
Ciudad             0
Anio               0
Kilometros         0
Combustible        0
Tipo_marchas       0
Mano               0
Consumo            0
Motor_CC           0
Potencia           0
Asientos           0
Descuento       1004
dtype: int64


In [7]:
train, test = na(train, test)

In [8]:
print(train.isna().sum())
print()
print(test.isna().sum())

Nombre          0
Ciudad          0
Anio            0
Kilometros      0
Combustible     0
Tipo_marchas    0
Mano            0
Consumo         0
Motor_CC        0
Potencia        0
Asientos        0
Precio_cat      0
dtype: int64

Nombre          0
Ciudad          0
Anio            0
Kilometros      0
Combustible     0
Tipo_marchas    0
Mano            0
Consumo         0
Motor_CC        0
Potencia        0
Asientos        0
dtype: int64


In [9]:
train

,Nombre,Ciudad,Anio,Kilometros,Combustible,Tipo_marchas,Mano,Consumo,Motor_CC,Potencia,Asientos,Precio_cat
0,Maruti Swift Dzire ZDI,G,2012.0,83000.0,Diesel,Manual,First,23.4 kmpl,1248 CC,74 bhp,5.0,3
1,Maruti Wagon R LXI Optional,I,2016.0,4800.0,Petrol,Manual,First,20.51 kmpl,998 CC,67.04 bhp,5.0,3
2,Mahindra KUV 100 mFALCON D75 K8,F,2016.0,26000.0,Diesel,Manual,First,25.32 kmpl,1198 CC,77 bhp,6.0,3
3,Hyundai i20 1.2 Magna,E,2013.0,56127.0,Petrol,Manual,First,18.5 kmpl,1197 CC,80 bhp,5.0,3
4,Honda Jazz 1.2 SV i VTEC,H,2017.0,41981.0,Petrol,Manual,First,18.7 kmpl,1199 CC,88.7 bhp,5.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
4813,Volkswagen Vento Petrol Highline,C,2012.0,67200.0,Petrol,Manual,First,15.04 kmpl,1598 CC,103.6 bhp,5.0,3
4814,Maruti Swift VDI,E,2014.0,83274.0,Diesel,Manual,First,22.9 kmpl,1248 CC,74 bhp,5.0,3
4815,Porsche Panamera Diesel 250hp,H,2014.0,60033.0,Diesel,Automatic,First,18.18 kmpl,2967 CC,250 bhp,4.0,5
4817,Hyundai Xcent 1.2 VTVT S,E,2015.0,65743.0,Petrol,Manual,First,20.14 kmpl,1197 CC,81.86 bhp,5.0,3


In [10]:
train, test = encodeNorm(train, test)

In [11]:
train

,Nombre,Ciudad,Anio,Kilometros,Combustible,Tipo_marchas,Mano,Consumo,Motor_CC,Potencia,Asientos,Precio_cat
0,18,5,2012.0,83000.0,1,1,0,23.40,1248.0,74.00,5.0,3
1,18,7,2016.0,4800.0,4,1,0,20.51,998.0,67.04,5.0,3
2,17,4,2016.0,26000.0,1,1,0,25.32,1198.0,77.00,6.0,3
3,10,3,2013.0,56127.0,4,1,0,18.50,1197.0,80.00,5.0,3
4,9,6,2017.0,41981.0,4,1,0,18.70,1199.0,88.70,5.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
4813,29,1,2012.0,67200.0,4,1,0,15.04,1598.0,103.60,5.0,3
4814,18,3,2014.0,83274.0,1,1,0,22.90,1248.0,74.00,5.0,3
4815,23,6,2014.0,60033.0,1,0,0,18.18,2967.0,250.00,4.0,5
4817,10,3,2015.0,65743.0,4,1,0,20.14,1197.0,81.86,5.0,3


In [12]:
train, label, test = preprocessing(train, test)

In [13]:
cross_val_score(model, train, label)

array([0.82022472, 0.81625   , 0.81125   , 0.81875   , 0.82625   ])

## Generar fichero de Kaggle

In [14]:
model.fit(train,label)
# Ahora predecimos
predict = model.predict(test)
predict = list(map(int,predict))
# Generamos 
df_result = pd.DataFrame({'id': test_ids, 'Precio_cat': predict})
df_result.to_csv(RESULTS+"try1.csv", index=False)